In [1]:
import sympy as sm
from sympy.physics.vector import init_vprinting
import numpy as np
import pandas as pd
from opty.direct_collocation import Problem, ConstraintCollocator
from opty.utils import parse_free

In [2]:
init_vprinting(use_latex='mathjax')

In [3]:
omega, zeta, k_delta, k_phi, k_phid, k_psi, t = sm.symbols('omega, zeta, k_delta, k_phi, k_dot_phi, k_psi, t') 
phi, delta, phid, deltad, psi, T_delta, Td_delta = [s(t) for s in sm.symbols('phi, delta, phi_d, delta_d, psi, T_delta, Td_delta', cls=sm.Function)]
F, psi_c = [s(t) for s in sm.symbols('F, psi_c', cls=sm.Function)]

In [4]:
x = sm.Matrix([phi, delta, phid, deltad, psi, T_delta, Td_delta])
x

⎡ φ  ⎤
⎢    ⎥
⎢ δ  ⎥
⎢    ⎥
⎢φ_d ⎥
⎢    ⎥
⎢δ_d ⎥
⎢    ⎥
⎢ ψ  ⎥
⎢    ⎥
⎢T_δ ⎥
⎢    ⎥
⎣Td_δ⎦

In [5]:
u = sm.Matrix([F, psi_c])
u

⎡ F ⎤
⎢   ⎥
⎣ψ_c⎦

In [6]:
A = sm.Matrix([[0, 0, 1, 0, 0, 0, 0],
               [0, 0, 0, 1, 0, 0, 0],
               [8.7171168 , -14.07978498,  -0.03225954,  -1.27710319, 0, 0, 0],
               [4.31150902, -18.75768104,   2.16708013,  -6.14451442, 0, 0, 0],
               [0, 3.7933839392202637, 0, 0.053529761278440759, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 1],
               [-omega**2 * k_delta * k_phid * k_phi,
                -omega**2 * k_delta,
                -omega**2 * k_delta * k_phid,
                0,
                -omega**2 * k_delta * k_phid * k_phi * k_psi,
                -omega**2,
                - 2 * omega * zeta]])
A

⎡        0                  0                 1                 0             
⎢                                                                             
⎢        0                  0                 0                 1             
⎢                                                                             
⎢    8.7171168         -14.07978498      -0.03225954       -1.27710319        
⎢                                                                             
⎢    4.31150902        -18.75768104      2.16708013        -6.14451442        
⎢                                                                             
⎢        0           3.79338393922026         0         0.0535297612784408    
⎢                                                                             
⎢        0                  0                 0                 0             
⎢                                                                             
⎢                 2            2                    

In [7]:
B = sm.Matrix([[0, 0],
               [0, 0],
               [0.00702997, 0],
               [-0.00357233, 0],
               [0, 0],
               [0, 0],
               [0, omega**2 * k_delta * k_phid * k_phi * k_psi]])
B

⎡     0                 0          ⎤
⎢                                  ⎥
⎢     0                 0          ⎥
⎢                                  ⎥
⎢0.00702997             0          ⎥
⎢                                  ⎥
⎢-0.00357233            0          ⎥
⎢                                  ⎥
⎢     0                 0          ⎥
⎢                                  ⎥
⎢     0                 0          ⎥
⎢                                  ⎥
⎢                                 2⎥
⎣     0       k_δ⋅k_dot_φ⋅kᵩ⋅k_ψ⋅ω ⎦

In [8]:
xdot = A * x + B * u
xdot

⎡                                                               φ_d           
⎢                                                                             
⎢                                                               δ_d           
⎢                                                                             
⎢                          0.00702997⋅F - 14.07978498⋅δ - 1.27710319⋅δ_d + 8.7
⎢                                                                             
⎢                         -0.00357233⋅F - 18.75768104⋅δ - 6.14451442⋅δ_d + 4.3
⎢                                                                             
⎢                                           3.79338393922026⋅δ + 0.05352976127
⎢                                                                             
⎢                                                              Td_δ           
⎢                                                                             
⎢                      2                         2  

In [9]:
eom = x.diff() - xdot
eom

⎡                                                               -φ_d + φ̇     
⎢                                                                             
⎢                                                               -δ_d + δ̇     
⎢                                                                             
⎢                         -0.00702997⋅F + 14.07978498⋅δ + 1.27710319⋅δ_d - 8.7
⎢                                                                             
⎢                         0.00357233⋅F + 18.75768104⋅δ + 6.14451442⋅δ_d - 4.31
⎢                                                                             
⎢                                           -3.79338393922026⋅δ - 0.0535297612
⎢                                                                             
⎢                                                             -Td_δ + T_̇δ    
⎢                                                                             
⎢                    2                         2    

# Load in the measured data

In [10]:
df = pd.read_csv('bicycle-measurements.csv')
df.mean()

Unnamed: 0    8.625000e+03
F            -1.235564e+00
T_delta      -2.567393e-01
Td_delta      2.360032e-02
delta         4.625490e-03
deltad        1.838603e-02
phi          -4.607070e-03
phid         -2.916783e-02
psi           1.497926e-08
time          4.312500e+01
v             4.247635e+00
dtype: float64

# Construct the problem

In [11]:
num_nodes = len(df)
num_nodes

17251

In [12]:
interval = df['time'].diff().mean()
interval

0.005

In [13]:
def obj(free):
    """Minimize the error in the angle, y1."""
    states, _, constants = parse_free(free, len(x), 0, num_nodes)
    j = (interval * np.sum((df['phi'] - states[0])**2) +
         interval * np.sum((df['delta'] - states[1])**2) +
         interval * np.sum((df['phid'] - states[2])**2) +
         interval * np.sum((df['deltad'] - states[3])**2) +
         interval * np.sum((df['psi'] - states[4])**2) +
         interval * np.sum((df['T_delta'] - states[5])**2) +
         interval * np.sum((df['Td_delta'] - states[6])**2))
    return j / 10000.0


def obj_grad(free):
    N = num_nodes
    states, _, constants = parse_free(free, len(x), 0, num_nodes)
    grad = np.zeros_like(free)
    grad[0 * N:1 * N] = 2.0 * interval * (states[0] - df['phi'])
    grad[1 * N:2 * N] = 2.0 * interval * (states[1] - df['delta'])
    grad[2 * N:3 * N] = 2.0 * interval * (states[2] - df['phid'])
    grad[3 * N:4 * N] = 2.0 * interval * (states[3] - df['deltad'])
    grad[4 * N:5 * N] = 2.0 * interval * (states[4] - df['psi'])
    grad[5 * N:6 * N] = 2.0 * interval * (states[5] - df['T_delta'])
    grad[6 * N:7 * N] = 2.0 * interval * (states[6] - df['Td_delta'])
    return grad / 10000.0

In [14]:
prob = Problem(obj,
               obj_grad,
               eom,
               x[:],
               num_nodes,
               interval,
               known_trajectory_map={F: df['F'].values,
                                     psi_c: np.zeros(num_nodes)},
              integration_method='midpoint')
prob.addOption('linear_solver', 'ma57')

In [15]:
state_guess = df[['phi', 'delta', 'phid', 'deltad', 'psi', 'T_delta', 'Td_delta']].values.T.flatten()
par_guess = np.array([20.0, -2.0, 3.0, 0.5, 30.0, 0.707])
initial_guess = np.hstack((state_guess, par_guess))

In [16]:
initial_guess.shape

(120763,)

In [17]:
7 * num_nodes

120757

In [18]:
prob.collocator.unknown_parameters

(k_δ, k_dot_φ, kᵩ, k_ψ, ω, ζ)

In [19]:
prob.num_free

120763

In [20]:
%timeit prob.con(initial_guess)

1000 loops, best of 3: 1.44 ms per loop


In [21]:
%timeit prob.con_jac(initial_guess)

100 loops, best of 3: 5.37 ms per loop


In [22]:
%timeit prob.obj(initial_guess)

100 loops, best of 3: 7.83 ms per loop


In [23]:
%timeit prob.obj_grad(initial_guess)

100 loops, best of 3: 5.46 ms per loop


In [24]:
solution, info = prob.solve(initial_guess)

In [25]:
solution[-6:]

array([  4.21109045e-03,  -7.46315727e+03,  -2.56454287e+00,
        -1.98214166e-02,   1.49002350e+01,   1.31316574e-01])

In [26]:
par_guess

array([ 20.   ,  -2.   ,   3.   ,   0.5  ,  30.   ,   0.707])

In [27]:
states, _, constants = parse_free(solution, len(x), 0, num_nodes)

In [28]:
constants

array([  4.21109045e-03,  -7.46315727e+03,  -2.56454287e+00,
        -1.98214166e-02,   1.49002350e+01,   1.31316574e-01])

In [29]:
#%matplotlib
import matplotlib.pyplot as plt

In [32]:
plt.plot(df['time'], df['phi'], '.', df['time'], states[0])

In [33]:
plt.show()